Bert属于，contextual word embeding 上下文词向量领域，也就是关注词的上下文。

对于语言模型来说，传统的n-gram模型的两个缺点：

    1、中心词的历史可能会很大，要看很长的句子
    2、词频统计的词，会有偏差，比如语料中，上海的词频大，那么上海就比北京的概率大，但是他们都是城市，那他们的权重应该是一样的。

然而，很多NLP任务是无监督的，毕竟监督的语料有限。

对于无监督训练来说，训练出来的词向量是无偏差的，较好的。

### word2vec

直接用神经网络去学习词向量。

特点：

    窗口 

### RNN

word2vec是没有考虑很长的上下文的，是无记忆功能的，而RNN就是为了解决这个问题，有个state神经元，记住前面的历史。

### LSTM GRU

RNN有个严重的问题，就是对于很长的状态使用梯度下降法传播时，会出现梯度消失或者梯度爆炸（梯度 > 1）。

LSTM 三个门、遗忘门、输入门，输出门

GRU 两个门 更新门、重置门

### seq2seq

序列到序列模型。

编码层 -> state 向量 -> 解码层

应用：

    翻译
    摘要
    问答

### Attention机制

#### 论文
《Deep Semantic Role Labeling with Self-Attention》

简单来说：这种新的构架替输入句的每个文字都创造一个context vector，而非仅仅替输入句创造一个从最终的hidden state得来的context vector，举例来说，如果一个输入句有N个文字，就会产生N个context vector，好处是，每个context vector能够被更有效的译码。

只关注某些具体部位，而不是全部整个句子。

### self-attention

attention model是如何运作的，缺点就是不能平行化，且忽略了输入句中文字间和目标句中文字间的关係。

前的Attention有两个致命的毛病 ，不能并行和它只能依赖单方向的序列（从前往后、从后往前），而无法看到后面的上下文

self-attention考虑到了整个句子，而不是单向的前面和后面的上下文。

其实就是用Q、K、V代替RNN，达到并行化的目的。


key、value的起源论文 Key-Value Memory Networks for Directly Reading Documents

### Transformer

https://arxiv.org/pdf/1706.03762.pdf 论文

https://blog.csdn.net/qq_41664845/article/details/84969266  图解Transformer

https://baijiahao.baidu.com/s?id=1627587324043258333&wfr=spider&for=pc  这篇

多层的encoder-decoder

self-attention层的输出向量z，其实已经含有了多语义信息，对吧，因为它考虑整个句子的其他部分。

multi-head  是指有多个Wq, Wk, Wv，这样能得到多个z。

位置编码：

    北京到上海的机票

    上海到北京的机票
    
    你好，我要北京到上海的机票
    
采用相对位置编码。

残差网络 和 LayerNorm

decode 多了一层普通的attention，编码层的输出 到 解码层。而且解码层只能看坐边，因为右边还没翻译出来。


“The transformer”在计算attention的方式有三种，

    1. encoder self attention，存在於encoder间.
    
    2. decoder self attention，存在於decoder间，
    
    3. encoder-decoder attention, 这种attention算法和过去的attention model相似
    
<img src="img/201ce8577815486295c841531d6a2fa6.jpeg">


### ELMO

18年提出 《Deep contextualized word representations》

思想：word embeding 考虑上下文，才能知道具体意思。

用contextual word embeding 作为特征，因为用无监督语料进行预训练，所以不适合特定任务

之前的做法的缺点是对于每一个单词都有唯一的一个embedding表示, 而对于多义词显然这种做法不符合直觉, 而单词的意思又和上下文相关。
ELMo的做法是我们只预训练language model, 而word embedding是通过输入的句子实时输出的, 这样单词的意思就是上下文相关的了, 这样就很大程度上缓解了歧义的发生。

ELMo的主要做法是先训练一个完整的语言模型，再用这个语言模型去处理需要训练的文本，生成相应的词向量，所以在文中一直强调ELMo的模型对同一个字在不同句子中能生成不同的词向量。它使用的是一个双向的LSTM语言模型，由一个前向和一个后向语言模型构成，目标函数就是取这两个方向语言模型的最大似然。

前向LSTM结构：

$p(t_1,t_2,...,t_N) = \prod^N_{k=1}p(t_k|t_1,t_2,...,t_{k-1})$

后向LSTM结构

$p(t_1,t_2,...,t_N) = \prod^N_{k=1}p(t_k|t_{k+1},t_{k+2},...,t_{N})$

最大似然函数：

$\sum_{k=1}^N(logp(t_k|t_1,t_2,...,t_{k-1}) + logp(t_k|t_{k+1},t_{k+2},...,t_{N}))$

在预训练好这个语言模型之后，ELMo就是根据公式来用作词表示，其实就是把这个双向语言模型的每一中间层进行一个求和。最简单的也可以使用最高层的表示来作为ELMo。然后在进行有监督的NLP任务时，可以将ELMo直接当做特征拼接到具体任务模型的词向量输入或者是模型的最高层表示上。


一般ELMo模型会在一个超大的语料库上进行预训练，因为是训练语言模型，不需要任何的标签，纯文本就可以，因而这里可以用超大的语料库，这一点的优势是十分明显的。训练完ELMo模型之后，就可以输入一个新句子，得到其中每个单词在当前这个句子上下文下的ELMo词向量了。

论文中提到，在训练的时候，发现使用合适的dropout和L2在ELMo模型上时会提升效果。

此时这个词向量就可以接入到下游的NLP任务中，比如问答、情感分析等。从接入的位置来看，可以与下游NLP任务本身输入的embedding拼接在一起，也可以与其输出拼接在一起。而从模型是否固定来看，又可以将ELMo词向量预先全部提取出来，即固定ELMo模型不让其训练，也可以在训练下游NLP任务时顺带fine-tune这个ELMo模型。总之，使用起来非常的方便，可以插入到任何想插入的地方进行增补。



大家有想过为什么ELMo的效果会比word2vec的效果好？我个人认为有一下几点：

    1、ELMo的假设前提一个词的词向量不应该是固定的，所以在一词多意方面ELMo的效果一定比word2vec要好。
    
    2、word2vec的学习词向量的过程是通过中心词的上下窗口去学习，学习的范围太小了，而ELMo在学习语言模型的时候是从整个语料库去学习的，而后再通过语言模型生成的词向量就相当于基于整个语料库学习的词向量，更加准确代表一个词的意思。
    
    3、ELMo还有一个优势，就是它建立语言模型的时候，可以运用非任务的超大语料库去学习，一旦学习好了，可以平行的运用到相似问题。

#### 解读地址

https://blog.csdn.net/Magical_Bubble/article/details/89160032

#### 源码地址

https://github.com/allenai/bilm-tf/tree/master/bilm

#### ELMO为啥能实现多层语义？

https://www.cnblogs.com/jiangxinyang/p/10060887.html

ELMO的提出就是为了解决这种语境问题，动态的去更新词的word embedding。ELMO的本质思想是：事先用语言模型在一个大的语料库上学习好词的word embedding，但此时的多义词仍然无法区分，不过没关系，我们接着用我们的训练数据（去除标签）来fine-tuning 预训练好的ELMO 模型。作者将这种称为domain transfer。这样利用我们训练数据的上下文信息就可以获得词在当前语境下的word embedding。

### OpenAI GPT

2018年OpenAi提出的论文《Improving Language Understanding by Generative Pre-Training》

#### 背景：
将训练好的词向量应用到特定任务中有两个主要挑战：

（1）it is unclear what type of optimization objectives are most effective at learning text representations that are useful for transfer

（2）there is no consensus on the most effective way to transfer these learned representations to the target task

而openAI GPT模型使用了一种半监督方式，它结合了无监督的预训练（pre-training）和有监督的微调（fine-tuning），旨在学习一种通用的表示方式，它转移到各种类型的NLP任务中都可以做很少的改变。

#### GPT 与 ELMO

https://zhuanlan.zhihu.com/p/49271699

#### 思想：
    
    1、根据任务 Fine-Tuning

    2、使用Transformer 代替RNN/LSTM，其实是只使用了Transformer的decode。

本文提出一种半监督的方式来处理语言理解的任务。使用非监督的预训练和监督方式的微调。我们的目标是学习一个通用的语言标示，可以经过很小的调整就应用在各种任务中。这个模型的设置不需要目标任务和非标注的数据集在同一个领域。模型有两个过程。

    1、使用语言模型学习一个深度模型
    2、随后，使用相应的监督目标将这些参数调整到目标任务
    


#### GPT与ELMO
GPT是“Generative Pre-Training”的简称，从名字看其含义是指的生成式的预训练。GPT也采用两阶段过程，第一个阶段是利用语言模型进行预训练，第二阶段通过Fine-tuning的模式解决下游任务。上图展示了GPT的预训练过程，其实和ELMO是类似的，主要不同在于两点：首先，特征抽取器不是用的RNN，而是用的Transformer，上面提到过它的特征抽取能力要强于RNN，这个选择很明显是很明智的；其次，GPT的预训练虽然仍然是以语言模型作为目标任务，但是采用的是单向的语言模型，所谓“单向”的含义是指：语言模型训练的任务目标是根据 [公式] 单词的上下文去正确预测单词 [公式] ， [公式] 之前的单词序列Context-before称为上文，之后的单词序列Context-after称为下文。ELMO在做语言模型预训练的时候，预测单词 [公式] 同时使用了上文和下文，而GPT则只采用Context-before这个单词的上文来进行预测，而抛开了下文。这个选择现在看不是个太好的选择，原因很简单，它没有把单词的下文融合进来，这限制了其在更多应用场景的效果，比如阅读理解这种任务，在做任务的时候是可以允许同时看到上文和下文一起做决策的。如果预训练时候不把单词的下文嵌入到Word Embedding中，是很吃亏的，白白丢掉了很多信息。

上面讲的是GPT如何进行第一阶段的预训练，那么假设预训练好了网络模型，后面下游任务怎么用？它有自己的个性，和ELMO的方式大有不同。
上图展示了GPT在第二阶段如何使用。首先，对于不同的下游任务来说，本来你可以任意设计自己的网络结构，现在不行了，你要向GPT的网络结构看齐，把任务的网络结构改造成和GPT的网络结构是一样的。然后，在做下游任务的时候，利用第一步预训练好的参数初始化GPT的网络结构，这样通过预训练学到的语言学知识就被引入到你手头的任务里来了，这是个非常好的事情。再次，你可以用手头的任务去训练这个网络，对网络参数进行Fine-tuning，使得这个网络更适合解决手头的问题。就是这样。看到了么？这有没有让你想起最开始提到的图像领域如何做预训练的过程（请参考上图那句非常容易暴露年龄的歌词）？对，这跟那个模式是一模一样的。

对于NLP各种花样的不同任务，怎么改造才能靠近GPT的网络结构呢？
<img src="img/v2-4c1dbed34a8f8469dc0fefe44b860edc_hd.jpg">

GPT论文给了一个改造施工图如上，其实也很简单：对于分类问题，不用怎么动，加上一个起始和终结符号即可；对于句子关系判断问题，比如Entailment，两个句子中间再加个分隔符即可；对文本相似性判断问题，把两个句子顺序颠倒下做出两个输入即可，这是为了告诉模型句子顺序不重要；对于多项选择问题，则多路输入，每一路把文章和答案选项拼接作为输入即可。从上图可看出，这种改造还是很方便的，不同任务只需要在输入部分施工即可。

#### 训练过程：

处理非监督文本(x1,x2,...,xm)的普通方法是用语言模型去最大化语言模型的极大似然。

$L_1(X)=\sum_ilogP(x_i|x_{i-k},...,x_{i-1}:\theta)$   可以从这里看出openAI GPT只考虑了从左到右的上下文

文章中使用的是多层Transformer的decoder的语言模型。这个多层的结构应用multi-headed self-attention在处理输入的文本加上位置信息的前馈网络，输出是词的概念分布。

#### 源码解读

https://blog.csdn.net/Magical_Bubble/article/details/89497002

GPT有什么值得改进的地方呢？其实最主要的就是那个单向语言模型，如果改造成双向的语言模型任务估计也没有Bert太多事了。当然，即使如此GPT也是非常非常好的一个工作，跟Bert比，其作者炒作能力亟待提升。


https://blog.csdn.net/Magical_Bubble/article/details/89524404 预训练模型杂谈

https://blog.csdn.net/Magical_Bubble/article/details/89497002 GPT

https://blog.csdn.net/Magical_Bubble/article/details/89514057 BERT

https://blog.csdn.net/Magical_Bubble/article/details/89499275 GPT2

https://blog.csdn.net/muumian123/article/details/84990765 预训练+fine tuning

其他
https://www.cnblogs.com/robert-dlut/p/9824346.html 
https://www.sohu.com/a/305221786_308467
https://www.jianshu.com/p/614a71c36bf1



### GTP-2

### Bert

#### 论文


Bert采用和GPT完全相同的两阶段模型，首先是语言模型预训练；其次是使用Fine-Tuning模式解决下游任务。和GPT的最主要不同在于在预训练阶段采用了类似ELMO的双向语言模型，当然另外一点是语言模型的数据规模要比GPT大。所以这里Bert的预训练过程不必多讲了。

#### 参考资料：

https://www.jiqizhixin.com/articles/2019-02-19-21

https://www.jiqizhixin.com/articles/2018-1-21 可视化bert

https://blog.csdn.net/muumian123/article/details/84990765  汇总解读